# Trabalho Prático 2 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Criptosistemas pós-quânticos PKE/KEM* - NTRU

Implementar o esquema NTRU em classes *Python/SageMath* apresentando, para cada um, as versões **KEM-IND-CPA** e **PKE-IND-CCA**.

Nota: Baseado no documento NTRU (https://ntru.org/f/ntru-20190330.pdf)

Conjuntos de parâmetros ntru-hrss específicos são denotados ntruhrss(n), por exemplo **ntruhrss701** é ntru-hrss com **n = 701**. O conjunto de parâmetros ntru-hrss recomendado é ntruhrss701, de acordo com o que foi submetido na 3ªronda do PQC.

### NTRU - KEM-IND-CPA



In [27]:
'''
Util para converter listas de inteiros para uma string que apresente esses inteiros por ordem correta
'''
def _toStr(intList):
    bin = BinaryStrings()
    s = [str(i) for i in intList]
    return bin.encoding("".join(s))
    

    
def _toBytes(salt):
    bts = [int(s).to_bytes( 1 , byteorder='big', signed=True ) for s in salt]
    res = b''.join(bts)
    return res

print(_toStr([1,2,5,7]))

print(_toBytes([1,2,5,7]))

#n = 701
#_Q.<x>  = Integers(3)[]
# Anel quociente Z[x]/(3,φn)
#phin = (x^n - 1)/(x-1)
#S3.<x>  = QuotientRing(_Q , _Q.ideal(phin))
#print(S3)
i = 5
N = 701
T = N // 4
u = [random.choice([-1,0,1]) for i in range(N)]
print(u)

00110001001100100011010100110111
b'\x01\x02\x05\x07'
[0, -1, 1, -1, 0, 0, 0, -1, 1, -1, 1, -1, -1, 0, -1, 0, -1, 0, 0, 0, 1, 0, 0, -1, 1, 1, 1, -1, -1, 0, -1, -1, 0, 1, 0, 0, 1, 1, 1, 0, 1, -1, 1, 0, 1, 0, 0, 0, 1, -1, 1, -1, -1, 1, 0, 1, -1, -1, 1, 0, 0, 1, -1, 0, 0, 0, 0, -1, 1, 0, -1, 1, 0, -1, 1, 0, 1, -1, 0, 1, 0, -1, 1, 0, 0, 0, -1, -1, -1, 0, -1, -1, 1, 1, 1, 1, 0, -1, 1, 1, 1, 1, 1, -1, 1, 0, 1, -1, 1, 0, 1, 0, 0, 1, -1, -1, -1, -1, 1, 0, -1, 0, 0, -1, 0, 1, -1, 1, 1, 1, 0, 0, -1, -1, -1, 1, -1, 0, 1, 1, -1, 1, 0, 1, 0, -1, -1, 1, 1, 0, 0, -1, 0, 1, -1, -1, -1, 1, 0, 1, -1, -1, 0, 1, 1, 1, -1, 0, 0, 0, 0, 0, -1, 0, 1, -1, -1, -1, 1, 0, 1, -1, 0, 0, 0, 0, 0, 0, -1, 1, -1, 0, 1, 0, 0, -1, -1, 1, 0, -1, 1, 0, -1, 0, 0, 1, 1, -1, 0, 0, -1, -1, 0, -1, -1, 1, 0, 1, 1, 1, 0, 0, -1, 1, 0, -1, -1, 0, 1, 0, -1, 0, -1, 1, 1, 0, -1, 0, 0, -1, -1, 1, 0, -1, 0, -1, -1, 1, 0, -1, -1, 0, 0, -1, -1, 1, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 1, -1, -1, 1, -1, 0, -1, 1, 0, 0, 1, 1, 1, -1, 1, 1, 1, -1, 0,

In [110]:
import random, os, hashlib

n = 701

# Converte uma sequencia de inteiros na sua representacao binaria
def randomBitString(size):
    
    # Gera uma sequencia de n bits aleatorios
    u = [random.choice([0,1]) for i in range(size)]
    # Mistura os valores da lista, so para aumentar a aleatoriedade
    random.shuffle(u)
    return u


def Ternary(bit_string):
    
    # S3 -> Anel quociente Z[x]/(3,φn)
    _Q.<x>  = Integers(3)[]
    phin = (x^n - 1)/(x-1)
    S3.<x>  = QuotientRing(_Q , _Q.ideal(phin))
    v = 0
    for i in range(n-1):
        
        v = v + sum([2^j*bit_string[8*i+j+1] for j in range(7)])*x^i
    
    return S3(v)
        

def Ternary_Plus(bit_string):
    
    # S3 -> Anel quociente Z[x]/(3,φn)
    _Q.<x>  = Integers(3)[]
    phin = (x^n - 1)/(x-1)
    S3.<x>  = QuotientRing(_Q , _Q.ideal(phin))
    
    # Set v = Ternary((b1; b2; ... ; bl))
    v = list(Ternary(bit_string))
    # t = somatorio de 0 ate n-2 de vi*vi+1
    t = sum([v[i]*v[i+1] for i in range(n-2)])
    # Set s = -1 if t < 0, otherwise set s = 1
    if t<0:
        s = -1
    else:
        s = 1
    # Set i = 0
    i = 0
    while i<n-1:
        # Set vi = s . vi
        v[i] = s*v[i]
        # Set i = i + 2
        i = i + 2
    
    return S3(v)
    

def Sample_fg(seed):
    
    # sample_iid_bits = 8*n - 8
    sample_iid_bits = 8*n - 8
    
    # Parse de fg_bits em f_bits||g_bits
    f_bits = seed[:sample_iid_bits]
    g_bits = seed[sample_iid_bits:]
    
    # Definir f = Ternary_Plus(f_bits)
    f = Ternary_Plus(f_bits)
    # Definir g0 = Ternary_Plus(g_bits)
    g0 = Ternary_Plus(g_bits)
    # Definir g = φ1.g0
    g = g0 * (x-1)
    
    return (f,g)


def Sample_rm(coins):
    
    # sample_iid_bits = 8*n - 8
    sample_iid_bits = 8*n - 8
    
    # Parse de rm_bits em r_bits||m_bits
    r_bits = coins[:sample_iid_bits]
    m_bits = coins[sample_iid_bits:]
    
    # Set r = Ternary(r_bits)
    r = Ternary(r_bits)
    # Set m = Ternary(m_bits)
    m = Ternary(m_bits)
    
    return (r,m)
    
    
# Precisamos de ter uma seed com 11200 bits, ou seja, 1400 bytes
#Sample_fg(randomBitString(11200))
#Sample_rm(randomBitString(11200))
#print(Sample_fg(randomBitString(11200)))

In [108]:
import hashlib

m = hashlib.sha3_256()
m.update(b"Nelson Faria")
m.digest()
#m.digest_size

b"\xce\xef\x01\x92%\xb4\x1d\xc4g\x9ca\xef\x02*\xa1<\x8a\x96Q\xf1B\xe7h\xa7Q\xa4'\xb4\xb9\xb9&\xf8"

In [ ]:
import random, hashlib

# Baseado no esquema da página 25 do documento https://ntru.org/f/ntru-20190330.pdf

class NTRU_KEM(object):
    
    def __init__(self, N, timeout=None):
        
        # Todas as inicializações de parâmetroa são baseadas na submissao com os parâmetros do NTRUHRSS-701, onde n = 701
        self.n = 701
        self.q = 8192
        self.p = 3
        self.sample_iid_bits = 5600
        
        # Definição dos aneis
        _Z.<x>  = ZZ[]
        # Anel quociente Z[x]/(φ1.φn)
        self.R.<x>   = QuotientRing(_Z , _Z.ideal(x^self.n - 1))
        _Q.<x>  = Integers(self.q)[]
        # Anel quociente Z[x]/(q,φ1.φn)
        self.Rq.<x>  = QuotientRing(_Q , _Q.ideal(x^self.n - 1))
        
          
    # Converte uma sequencia de inteiros na sua representacao binaria
    def randomBitString(self, size):
        
        # Gera uma sequencia de n bits aleatorios
        u = [random.choice([0,1]) for i in range(size)]
        # Mistura os valores da lista, so para aumentar a aleatoriedade
        random.shuffle(u)
        return u
    
    
    # Gera um polinomio ternario 
    def Ternary(self, bit_string):
        
        # S3 -> Anel quociente Z[x]/(3,φn)
        _Q.<x>  = Integers(3)[]
        phin = (x^self.n - 1)/(x-1)
        S3.<x>  = QuotientRing(_Q , _Q.ideal(phin))
        v = 0
        for i in range(self.n-1):
            
            v = v + sum([2^j*bit_string[8*i+j+1] for j in range(7)])*x^i
        
        return S3(v)
    
    
    # Gera um polinomio ternario que satisfaz a non-negative correlation property
    def Ternary_Plus(self, bit_string):
        
        # S3 -> Anel quociente Z[x]/(3,φn)
        _Q.<x>  = Integers(3)[]
        phin = (x^n - 1)/(x-1)
        S3.<x>  = QuotientRing(_Q , _Q.ideal(phin))
        
        # Set v = Ternary((b1; b2; ... ; bl))
        v = list(self.Ternary(bit_string))
        # t = somatorio de 0 ate n-2 de vi*vi+1
        t = sum([v[i]*v[i+1] for i in range(n-2)])
        # Set s = -1 if t < 0, otherwise set s = 1
        if t<0:
            s = -1
        else:
            s = 1
            
        # Set i = 0
        i = 0
        while i<self.n-1:
            # Set vi = s . vi
            v[i] = s*v[i]
            # Set i = i + 2
            i = i + 2
        
        return S3(v)
    
    
    # Gera um polinomio f em Lf (no nosso caso, em T+) e um polinomio g em Lg (no nosso caso, em {φ1.v : v € T+})
    def Sample_fg(seed):
        
        # sample_iid_bits = 8*n - 8
        sample_iid_bits = 8*self.n - 8
        
        # Parse de fg_bits em f_bits||g_bits
        f_bits = seed[:sample_iid_bits]
        g_bits = seed[sample_iid_bits:]
        
        # Definir f = Ternary_Plus(f_bits)
        f = self.Ternary_Plus(f_bits)
        # Definir g0 = Ternary_Plus(g_bits)
        g0 = self.Ternary_Plus(g_bits)
        # Definir g = φ1.g0
        g = g0 * (x-1)
        
        return (f,g)
    
    
    # Gera um polinomio r em Lr (no nosso caso, em T) e um polinomio m em Lm (no nosso caso, em T)
    def Sample_rm(self, coins):
        
        # sample_iid_bits = 8*n - 8
        sample_iid_bits = 8*self.n - 8
        
        # Parse de rm_bits em r_bits||m_bits
        r_bits = coins[:sample_iid_bits]
        m_bits = coins[sample_iid_bits:]
        
        # Set r = Ternary(r_bits)
        r = self.Ternary(r_bits)
        # Set m = Ternary(m_bits)
        m = self.Ternary(m_bits)
        
        return (r,m)
    
    
    
    
    
    # Funcao usada para gerar o par de chaves pública e privada
    def geraChaves(n=N, t=T, q=Q):
        
        
        while True:
            F = _fromS()
            G = _fromS()
            f = 1 + 3 * _toR(F)
            g = 3 * _toR(G)
            (flag, h) = _h(f,g)
            if flag:
                break
        return {'sk' : f , 'pk' : h}

    
    # Funcao que serve para encapsular a chave que for acordada a partir de uma chave publica
    def encapsula(self, e, n):
        
    
    
    # Funcao usada para desencapsular uma chave, a partir do seu "encapsulamento"
    def desencapsula(self, cs):
        
        
        